In [10]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU,Dropout2d
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds_log.npy')
means = np.load('../Data/means_log.npy')
long_normal_value = np.load('../Data/long_lognormal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = .5

In [5]:
# baseline model
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 106.31
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# np.mean(np.square(y - yhat))
# >>> 1.1114173

In [5]:
# set up data loader
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalized(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalized(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [21]:
conv = Sequential(ConvBatchRelu1D(3,8,3),
                  Dropout2d(),
                  ConvBatchRelu1D(8,16,3),
                  Dropout2d(),
                  ConvBatchRelu1D(16,32,3),
                  Dropout2d(),
                  ConvBatchRelu1D(32,64,3),
                  Dropout2d())

In [22]:
rnn = GRU(64,64,2,batch_first=True,dropout=0.5,bidirectional=True)

In [23]:
linear = Sequential(Linear(128,64),LeakyReLU(0.1,True),Dropout(0.5),Linear(64,1))

In [24]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized(conv,rnn,linear).to('cuda:0')

In [25]:
loss_func_nor = loss_func_generator_normalized(MSELoss())
# loss_func_nor = loss_func_generator_normalized(L1Loss())
#loss_func_nor = loss_func_generator_normalized(SMAPE)

In [26]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [27]:
model_nor = fit(2, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip)

epoch:0, train_loss:0.8411190489413207, val_loss:0.8890881538391113
epoch:1, train_loss:0.7849993703742102, val_loss:0.8491793870925903
Training completed in 88.57397508621216s


In [28]:
model = CNN_RNN2seq_logNormalized(conv,rnn,linear).to('cuda:0')

In [29]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [30]:
loss_func = loss_func_generator(SMAPE)

In [31]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:38.098459760636246, val_loss:37.665523529052734
epoch:1, train_loss:37.87765685066658, val_loss:37.63193893432617
epoch:2, train_loss:37.83407561840789, val_loss:37.53418731689453
Training completed in 159.24003839492798s


In [32]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.647764003338594, val_loss:37.22309875488281
epoch:1, train_loss:37.62140609958629, val_loss:37.2067985534668
epoch:2, train_loss:37.54973493585933, val_loss:37.518653869628906
Training completed in 159.79315328598022s


In [33]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.53409971424953, val_loss:37.27890396118164
epoch:1, train_loss:37.50200062193401, val_loss:37.45276641845703
epoch:2, train_loss:37.40306583760316, val_loss:37.62301254272461
Training completed in 159.00133299827576s
